<a href="https://colab.research.google.com/github/AhlamBashiti1/MedCUI_ML_Project/blob/main/Text_Model_BioBert_Clinical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 58.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requ

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

# Load your data
train_df = pd.read_csv('/content/Train_with_clusters.csv')
val_df = pd.read_csv('/content/Val_with_clusters.csv')
test_df = pd.read_csv('/content/Test_with_clusters.csv')

label_cols = ['0', '1', '2', '3']
y_train = train_df[label_cols].values
y_val = val_df[label_cols].values
y_test = test_df[label_cols].values
all_captions = pd.concat([train_df['Caption'], val_df['Caption']])


In [ ]:
# TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_vectorizer.fit(all_captions)

X_train_tfidf = tfidf_vectorizer.transform(train_df['Caption'])
X_val_tfidf = tfidf_vectorizer.transform(val_df['Caption'])
X_test_tfidf = tfidf_vectorizer.transform(test_df['Caption'])

# Word2Vec
sentences = [cap.lower().split() for cap in all_captions]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4, epochs=20)

def get_w2v_features(captions, model):
    features = []
    for cap in captions:
        words = cap.lower().split()
        vecs = [model.wv[word] for word in words if word in model.wv]
        features.append(np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size))
    return np.array(features)

X_train_w2v = get_w2v_features(train_df['Caption'], w2v_model)
X_val_w2v = get_w2v_features(val_df['Caption'], w2v_model)
X_test_w2v = get_w2v_features(test_df['Caption'], w2v_model)

# Combine TF-IDF + Word2Vec
X_train_comb = hstack([X_train_tfidf, csr_matrix(X_train_w2v)])
X_val_comb = hstack([X_val_tfidf, csr_matrix(X_val_w2v)])
X_test_comb = hstack([X_test_tfidf, csr_matrix(X_test_w2v)])


In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm


In [ ]:
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import torch
import numpy as np

# Load BioBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
bert_model = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
bert_model.eval()

def get_bert_embeddings(texts, tokenizer, model, max_len=64):
    embeddings = []
    with torch.no_grad():
        for txt in tqdm(texts):
            inputs = tokenizer(txt, return_tensors='pt', truncation=True, padding='max_length', max_length=max_len)
            outputs = model(**inputs)
            cls_embed = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
            embeddings.append(cls_embed)
    return np.array(embeddings)

# Generate BioBERT embeddings for train, val, and test captions
X_train_bert = get_bert_embeddings(train_df['Caption'], tokenizer, bert_model)
X_val_bert = get_bert_embeddings(val_df['Caption'], tokenizer, bert_model)
X_test_bert = get_bert_embeddings(test_df['Caption'], tokenizer, bert_model)



config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

  0%|          | 0/59957 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

100%|██████████| 9927/9927 [09:49<00:00, 16.83it/s]


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier


In [ ]:
##################################Only for bert
from sklearn.preprocessing import StandardScaler
# normalize all

scaler = StandardScaler()
X_train_bert = scaler.fit_transform(X_train_bert)
X_val_bert = scaler.transform(X_val_bert)
X_test_bert = scaler.transform(X_test_bert)

In [ ]:
#######################################Only for bert
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

# ✅ Define evaluation function for multi-label classification
def evaluate(model, X, y_true):
    y_pred = model.predict(X)
    return {
        "f1_micro": f1_score(y_true, y_pred, average='micro'),
        "f1_macro": f1_score(y_true, y_pred, average='macro'),
        "precision_micro": precision_score(y_true, y_pred, average='micro', zero_division=0),
        "recall_micro": recall_score(y_true, y_pred, average='micro'),
        "accuracy": accuracy_score(y_true, y_pred)
    }

# ✅ Feature sets (train, val, test tuples)
feature_sets = {
    'BioBert': (X_train_bert, X_val_bert, X_test_bert)
}

# ✅ Classifiers and params suited for ClinicalBERT features
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier

models = {
    'LogisticRegression': OneVsRestClassifier(LogisticRegression(solver='liblinear', max_iter=1000)),
    'LinearSVC': OneVsRestClassifier(LinearSVC(max_iter=2000)),
    'SGDClassifier': OneVsRestClassifier(SGDClassifier(loss="log_loss", max_iter=1000)),
    'MLPClassifier': OneVsRestClassifier(MLPClassifier(hidden_layer_sizes=(256,), max_iter=300))
}

params = {
    'LogisticRegression': {
        'estimator__C': [0.1, 1, 10]
    },
    'LinearSVC': {
        'estimator__C': [0.1, 1, 10]
    },
    'SGDClassifier': {
       'estimator__alpha': [1e-4, 1e-3, 1e-2]
    },
    'MLPClassifier': {
        'estimator__hidden_layer_sizes': [(256,), (512,)],
       'estimator__alpha': [0.0001, 0.001]
   }
}

results = []

# ✅ Loop over feature sets and classifiers
for feat_name, (X_tr, X_va, X_te) in tqdm(feature_sets.items(), desc="🔁 Feature Sets"):
    for model_name, base_model in tqdm(models.items(), desc=f"⚙️ Models for {feat_name}", leave=False):
        print(f"\n🔍 Running GridSearchCV for {model_name} with {feat_name}")

        grid = GridSearchCV(base_model, params[model_name], scoring='f1_micro', cv=2, n_jobs=-1, verbose=1)
        grid.fit(X_tr, y_train)

        # ✅ Validation evaluation
        val_scores = evaluate(grid.best_estimator_, X_va, y_val)
        val_scores.update({
            "model": model_name,
            "features": feat_name,
            "split": "val",
            "best_params": grid.best_params_
        })
        results.append(val_scores)

        # ✅ Final test evaluation (no retraining)
        test_scores = evaluate(grid.best_estimator_, X_te, y_test)
        test_scores.update({
            "model": model_name,
            "features": feat_name,
            "split": "test",
            "best_params": grid.best_params_
        })
        results.append(test_scores)

        print(f"✅ Completed: {model_name} with {feat_name}")
        print(f"📊 Validation F1: {val_scores['f1_micro']:.4f} | Test F1: {test_scores['f1_micro']:.4f}")


⚙️ Models for ClinicalBERT:   0%|          | 0/4 [00:00<?, ?it/s]


🔍 Running GridSearchCV for LogisticRegression with ClinicalBERT
Fitting 2 folds for each of 3 candidates, totalling 6 fits



⚙️ Models for ClinicalBERT:  25%|██▌       | 1/4 [03:57<11:51, 237.08s/it]

✅ Completed: LogisticRegression with ClinicalBERT
📊 Validation F1: 0.8358 | Test F1: 0.8358

🔍 Running GridSearchCV for LinearSVC with ClinicalBERT
Fitting 2 folds for each of 3 candidates, totalling 6 fits



⚙️ Models for ClinicalBERT:  50%|█████     | 2/4 [07:23<07:17, 218.81s/it]

✅ Completed: LinearSVC with ClinicalBERT
📊 Validation F1: 0.8349 | Test F1: 0.8359

🔍 Running GridSearchCV for SGDClassifier with ClinicalBERT
Fitting 2 folds for each of 3 candidates, totalling 6 fits



⚙️ Models for ClinicalBERT:  75%|███████▌  | 3/4 [09:08<02:47, 167.15s/it]

✅ Completed: SGDClassifier with ClinicalBERT
📊 Validation F1: 0.8293 | Test F1: 0.8324

🔍 Running GridSearchCV for MLPClassifier with ClinicalBERT
Fitting 2 folds for each of 4 candidates, totalling 8 fits



🔁 Feature Sets: 100%|██████████| 1/1 [21:26<00:00, 1286.53s/it]

✅ Completed: MLPClassifier with ClinicalBERT
📊 Validation F1: 0.8331 | Test F1: 0.8371


In [ ]:
models = {
    #'LinearSVC': OneVsRestClassifier(LinearSVC(max_iter=2000)),
   'RandomForest': MultiOutputClassifier(RandomForestClassifier()), #very slow
    #'AdaBoost': MultiOutputClassifier(AdaBoostClassifier()),#next slow
   #'LogisticRegression': OneVsRestClassifier(LogisticRegression(solver='liblinear'))
}

params = {
    #'LinearSVC': {'estimator__C': [0.1, 1, 10]}
     'RandomForest': {
        'estimator__n_estimators': [100, 200],
        'estimator__max_depth': [10, 20, None]
     },
     #'AdaBoost': {'estimator__n_estimators': [50, 100]},
     #'LogisticRegression': {'estimator__C': [0.1, 1, 10]}
}

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm  #

def evaluate(model, X, y_true):
    y_pred = model.predict(X)
    return {
        "f1_micro": f1_score(y_true, y_pred, average='micro'),
        "precision_micro": precision_score(y_true, y_pred, average='micro', zero_division=0),
        "recall_micro": recall_score(y_true, y_pred, average='micro'),
        "accuracy": accuracy_score(y_true, y_pred)
    }

feature_sets = {
    'TFIDF': (X_train_tfidf, X_val_tfidf),
    'W2V': (X_train_w2v, X_val_w2v),
    'TFIDF+W2V': (X_train_comb, X_val_comb),
    # 'ClinicalBERT': (X_train_bert, X_val_bert)
}

results = []

# loop over feature sets with tqdm
for feat_name, (X_tr, X_va) in tqdm(feature_sets.items(), desc="Feature Sets"):
    # loop over models with tqdm
    for model_name, base_model in tqdm(models.items(), desc=f"Models for {feat_name}", leave=False):
        print(f"🔍 Running GridSearchCV for {model_name} with {feat_name}")
        grid = GridSearchCV(base_model, params[model_name], scoring='f1_micro', cv=2, n_jobs=-1, verbose=1)
        grid.fit(X_tr, y_train)

        scores = evaluate(grid.best_estimator_, X_va, y_val)
        scores.update({
            "model": model_name,
            "features": feat_name,
            "best_params": grid.best_params_
        })
        results.append(scores)


Models for TFIDF:   0%|          | 0/1 [00:00<?, ?it/s]

🔍 Running GridSearchCV for LogisticRegression with TFIDF
Fitting 2 folds for each of 3 candidates, totalling 6 fits



Models for W2V:   0%|          | 0/1 [00:00<?, ?it/s]

🔍 Running GridSearchCV for LogisticRegression with W2V
Fitting 2 folds for each of 3 candidates, totalling 6 fits



Models for TFIDF+W2V:   0%|          | 0/1 [00:00<?, ?it/s]

🔍 Running GridSearchCV for LogisticRegression with TFIDF+W2V
Fitting 2 folds for each of 3 candidates, totalling 6 fits



Feature Sets: 100%|██████████| 3/3 [06:31<00:00, 130.46s/it]


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

# ✅ Define evaluation function
def evaluate(model, X, y_true):
    y_pred = model.predict(X)
    return {
        "f1_micro": f1_score(y_true, y_pred, average='micro'),
        "precision_micro": precision_score(y_true, y_pred, average='micro', zero_division=0),
        "recall_micro": recall_score(y_true, y_pred, average='micro'),
        "accuracy": accuracy_score(y_true, y_pred)
    }

# ✅ Feature sets (train, val, test tuples)
feature_sets = {
    #'TFIDF': (X_train_tfidf, X_val_tfidf, X_test_tfidf),
    #'W2V': (X_train_w2v, X_val_w2v, X_test_w2v),
    #'TFIDF+W2V': (X_train_comb, X_val_comb, X_test_comb),
     'ClinicalBERT': (X_train_bert, X_val_bert, X_test_bert)
}

results = []

# ✅ Loop over features and models
for feat_name, (X_tr, X_va, X_te) in tqdm(feature_sets.items(), desc="🔁 Feature Sets"):
    for model_name, base_model in tqdm(models.items(), desc=f"⚙️ Models for {feat_name}", leave=False):
        print(f"\n🔍 Running GridSearchCV for {model_name} with {feat_name}")

        grid = GridSearchCV(base_model, params[model_name], scoring='f1_micro', cv=2, n_jobs=-1, verbose=1)
        grid.fit(X_tr, y_train)

        # ✅ Validation evaluation
        val_scores = evaluate(grid.best_estimator_, X_va, y_val)
        val_scores.update({
            "model": model_name,
            "features": feat_name,
            "split": "val",
            "best_params": grid.best_params_
        })
        results.append(val_scores)

        # ✅ Final test evaluation (no retraining)
        test_scores = evaluate(grid.best_estimator_, X_te, y_test)
        test_scores.update({
            "model": model_name,
            "features": feat_name,
            "split": "test",
            "best_params": grid.best_params_
        })
        results.append(test_scores)

        print(f"✅ Completed: {model_name} with {feat_name}")
        print(f"📊 Validation F1: {val_scores['f1_micro']:.4f} | Test F1: {test_scores['f1_micro']:.4f}")


⚙️ Models for TFIDF:   0%|          | 0/1 [00:00<?, ?it/s]


🔍 Running GridSearchCV for RandomForest with TFIDF
Fitting 2 folds for each of 6 candidates, totalling 12 fits



🔁 Feature Sets:  50%|█████     | 1/2 [33:32<33:32, 2012.21s/it]

✅ Completed: RandomForest with TFIDF
📊 Validation F1: 0.8834 | Test F1: 0.8875



⚙️ Models for W2V:   0%|          | 0/1 [00:00<?, ?it/s]


🔍 Running GridSearchCV for RandomForest with W2V
Fitting 2 folds for each of 6 candidates, totalling 12 fits



🔁 Feature Sets: 100%|██████████| 2/2 [1:35:07<00:00, 2853.68s/it]

✅ Completed: RandomForest with W2V
📊 Validation F1: 0.8367 | Test F1: 0.8408


In [ ]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.to_csv("BioBertTfourclassifier.csv", index=False)
results_df


,f1_micro,f1_macro,precision_micro,recall_micro,accuracy,model,features,split,best_params
0,0.835847,0.751534,0.842823,0.828986,0.432855,LogisticRegression,ClinicalBERT,val,{'estimator__C': 0.1}
1,0.835825,0.751787,0.841754,0.829979,0.431550,LogisticRegression,ClinicalBERT,test,{'estimator__C': 0.1}
2,0.834949,0.742666,0.845218,0.824926,0.431240,LinearSVC,ClinicalBERT,val,{'estimator__C': 1}
3,0.835898,0.744939,0.844217,0.827742,0.433061,LinearSVC,ClinicalBERT,test,{'estimator__C': 1}
4,0.829301,0.737321,0.834264,0.824396,0.412763,SGDClassifier,ClinicalBERT,val,{'estimator__alpha': 0.01}
5,0.832396,0.740794,0.834339,0.830461,0.421779,SGDClassifier,ClinicalBERT,test,{'estimator__alpha': 0.01}
6,0.833100,0.768433,0.837391,0.828854,0.431543,MLPClassifier,ClinicalBERT,val,"{'estimator__alpha': 0.0001, 'estimator__hidde..."
7,0.837142,0.770951,0.840472,0.833838,0.443034,MLPClassifier,ClinicalBERT,test,"{'estimator__alpha': 0.0001, 'estimator__hidde..."
